<a href="https://colab.research.google.com/github/GwenTsang/Syntaxe/blob/main/Hopsparser_CamemBERT_Occitan_TTB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# On télécharge depuis ce lien car c'est plus rapide
import kagglehub
path = kagglehub.dataset_download("gwendaltsang/ud-french-gsd-2-9-camembert")

100%|██████████| 729M/729M [00:13<00:00, 58.3MB/s]

Extracting files...


In [2]:
import shutil
print(path)

dest = "/content/ud-french-gsd-camembert"

# déplacement
shutil.move(path, dest)

print("Déplacé vers :", dest)

/root/.cache/kagglehub/datasets/gwendaltsang/ud-french-gsd-2-9-camembert/versions/1
Déplacé vers : /content/ud-french-gsd-camembert


In [3]:
# Installations

!pip uninstall -y torch torchvision torchaudio numpy transformers hopsparser tokenizers 2>/dev/null || true

!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121

# hopsparser needs >=4.19
!pip install 'transformers>=4.41.0,<4.52.0'

print("Installation complete!")
print("")
print("Restart runtime")

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 129.7 MB/s eta 0:00:00
     ━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 110.6 MB/s eta 0:00:00
Installation complete!

Restart runtime


In [4]:
!pip install hopsparser -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 6.2 MB/s eta 0:00:00


In [5]:
# Verification
import numpy as np
print(f"NumPy: {np.__version__}")

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

import transformers
print(f"Transformers: {transformers.__version__}")

import hopsparser
print("Hopsparser: ✓ OK")

NumPy: 2.0.2
PyTorch: 2.5.1+cu121
CUDA available: True
GPU: Tesla T4
Transformers: 4.51.3
Hopsparser: ✓ OK


In [6]:
# Extract model
!tar -xf "/content/ud-french-gsd-camembert/UD_French-FTB-camembert.tar.xz"
!ls -la /content/UD_French-FTB-camembert/

total 853744
drwxr-xr-x 3 1004 1004      4096 Mar  5  2023 .
drwxr-xr-x 1 root root      4096 Dec 19 13:55 ..
-rw-r--r-- 1 1004 1004       748 Mar  5  2023 config.json
drwxr-xr-x 3 1004 1004      4096 Mar  5  2023 lexers
-rw-r--r-- 1 1004 1004 874212223 Mar  6  2023 weights.pt


In [7]:
# Download conllu file
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_Occitan-TTB/master/oc_ttb-ud-test.conllu
!echo "Test file downloaded: $(wc -l < oc_ttb-ud-test.conllu) lines"

Test file downloaded: 12097 lines


# **Inférence**

In [8]:
# Run hopsparser
!hopsparser parse /content/UD_French-FTB-camembert /content/oc_ttb-ud-test.conllu /content/oc_ttb-ud-out.conllu

2025-12-19 13:56:29.828148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766152590.118886    3308 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766152590.199866    3308 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766152590.798734    3308 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766152590.798787    3308 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766152590.798796    3308 computation_placer.cc:177] computation placer alr

In [9]:
!echo "Test file downloaded: $(wc -l < oc_ttb-ud-out.conllu) lines"

Test file downloaded: 12097 lines


# **Entraînement**

In [11]:
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_Occitan-TTB/refs/heads/master/oc_ttb-ud-dev.conllu
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_Occitan-TTB/refs/heads/master/oc_ttb-ud-train.conllu
!wget -q https://raw.githubusercontent.com/hopsparser/hopsparser/refs/heads/main/examples/configs/camembert.yaml

In [13]:
!hopsparser train /content/camembert.yaml /content/oc_ttb-ud-train.conllu /content/UD_French-FTB-camembert --dev-file /content/oc_ttb-ud-dev.conllu --test-file /content/oc_ttb-ud-test.conllu

2025-12-19 14:04:05.691864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766153045.712523    5238 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766153045.718613    5238 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766153045.734922    5238 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766153045.734951    5238 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766153045.734955    5238 computation_placer.cc:177] computation placer alr